In [1]:
import re
import os
import pandas as pd
import numpy as np

In [2]:
# Change directory
directoryPath = os.getenv('PWD')
directoryName = 'Documents/Python/Github/Data/SA'
directoryPath = os.path.join(directoryPath, directoryName)
os.chdir(directoryPath)

<h3>I need to read data from multiple spreadsheets. Each spreadsheet has multiple sheets with various columns.<h3>

In [3]:
# Specify the columns I will keep in the final dataframe
cols_to_keep = ['Platform', 'SocialNetwork', 'SenderScreenName', 'Message', 'CreatedTime', 'Sentiment']
# Create an empty dataframe to hold the data
df = pd.DataFrame(columns=cols_to_keep)

sheet_to_df_map = {}
# Look for files that satisfy the criteria
for f in [f for f in os.listdir(directoryPath) if f[-4:] == 'xlsx' and 'Inbound' in f]:
    xls = pd.ExcelFile(f)
    # Iterate through each sheet
    for sheet_name in xls.sheet_names:
        # Store sheet name as key, dataframe as value 
        sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
        # Create a column 'Platform' to store sheet name 
        sheet_to_df_map[sheet_name]['Platform'] = sheet_name.strip()
        # Standardize platform names - resulting from random sheet names
        sheet_to_df_map[sheet_name]['Platform'] = sheet_to_df_map[sheet_name]['Platform'].replace({
                           'FB - PMs':'FB - Private Messages', 
                           'Twitter Mentions': 'Twitter - Mentions'       
                           })
        # Only keep the columns I need 
        sheet_to_df_map[sheet_name] = sheet_to_df_map[sheet_name][cols_to_keep]
        # Combine all dataframes into a single one
        df = pd.concat([df,sheet_to_df_map[sheet_name]])

# Remove duplicate records from combining various spreadsheets
df = df.drop_duplicates()
df = df.reset_index(drop=True)

# Create a date column based on the timestamp
df['Date'] = df['CreatedTime'].apply(lambda x: x.date())

In [4]:
df.head()

,Platform,SocialNetwork,SenderScreenName,Message,CreatedTime,Sentiment,Date
0,FB - Wall Posts,FACEBOOK,Ann Heacock Sinton,Why isn't the online ordering working? I went...,2020-03-23 17:08:33,NEGATIVE,2020-03-23
1,FB - Wall Posts,FACEBOOK,Ari Graves,This is just greedy. \n\n$3.50 for this? \n\nM...,2020-03-19 01:33:58,NEGATIVE,2020-03-19
2,FB - Wall Posts,FACEBOOK,BL Schafer,Your company really needs an app like Sams Clu...,2020-03-18 17:35:08,NEGATIVE,2020-03-18
3,FB - Wall Posts,FACEBOOK,Bob Galivan,I was in the Beachwood store yesterday. NOT ON...,2020-03-21 12:28:26,NEGATIVE,2020-03-21
4,FB - Wall Posts,FACEBOOK,Bob Varisco,I posted Wednesday about the fact that I'm at ...,2020-03-27 16:29:03,NEGATIVE,2020-03-27


In [5]:
# Convert HTML Character Entities
replace_dict = {'&#39;': '\'',
                '&quot;': '\"',
                '&amp;': '&',
                '&lt;': '<',
                '&gt;': '>',
                '\n': ''}

for k,v in replace_dict.items():
    df['Message'] = df['Message'].apply(lambda x: re.sub(k, v, x) if pd.isnull(x)==False else np.nan)

In [6]:
hashtag_pattern = r'\B#\w*[a-zA-Z]+\w*'
mention_pattern = r'\B@\w*[a-zA-Z]+\w*'
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|\w+(?:\w+|\.)\w+\.(?:com|ly)\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
covid_pattern = r'(virus|corona|mask|glove|disinfect|cough|sanitize|pandemic|covid|infect|sanitizing|cov19|wipe)'

In [7]:
# Extract hashtags, mentions and urls
df['hashtag'] = df['Message'].apply(lambda x: re.findall(hashtag_pattern, x.lower()) if pd.isnull(x)==False else np.nan)
df['mention'] = df['Message'].apply(lambda x: re.findall(mention_pattern, x.lower()) if pd.isnull(x)==False else np.nan)
df['url'] = df['Message'].apply(lambda x: re.findall(url_pattern, x) if pd.isnull(x)==False else np.nan)

In [8]:
# Function to find all covid keywords
def find_covid_words(string):
    if pd.isnull(string) == False:
        return re.findall(covid_pattern, string.lower())
    else:
        return []

covid_keywords = df['Message'].apply(find_covid_words)
df['Covid_Keywords'] = covid_keywords.apply(lambda x: set(x)) 
df['Covid_Keywords_Count'] = df['Covid_Keywords'].apply(lambda x: len(x))

# Flag when there are at least two key words
df['Covid_Flag'] = df['Covid_Keywords_Count'].apply(lambda x: 1 if x>1 else 0)

df.head()

,Platform,SocialNetwork,SenderScreenName,Message,CreatedTime,Sentiment,Date,hashtag,mention,url,Covid_Keywords,Covid_Keywords_Count,Covid_Flag
0,FB - Wall Posts,FACEBOOK,Ann Heacock Sinton,Why isn't the online ordering working? I went...,2020-03-23 17:08:33,NEGATIVE,2020-03-23,[],[],[],{},0,0
1,FB - Wall Posts,FACEBOOK,Ari Graves,This is just greedy. $3.50 for this? Making mo...,2020-03-19 01:33:58,NEGATIVE,2020-03-19,[],[],[],{},0,0
2,FB - Wall Posts,FACEBOOK,BL Schafer,Your company really needs an app like Sams Clu...,2020-03-18 17:35:08,NEGATIVE,2020-03-18,[],[],[],"{corona, virus}",2,1
3,FB - Wall Posts,FACEBOOK,Bob Galivan,I was in the Beachwood store yesterday. NOT ON...,2020-03-21 12:28:26,NEGATIVE,2020-03-21,[],[],[],"{mask, covid, virus, glove}",4,1
4,FB - Wall Posts,FACEBOOK,Bob Varisco,I posted Wednesday about the fact that I'm at ...,2020-03-27 16:29:03,NEGATIVE,2020-03-27,[],[],[],{},0,0


In [9]:
df['cleanMsg'] = df['Message']

# Remove all hashtags, mentions and urls from the main messages
for p in [hashtag_pattern, mention_pattern, url_pattern]:
    df['cleanMsg'] = df['cleanMsg'].apply(lambda x: re.sub(p, ' ', x) if pd.isnull(x)==False else np.nan)

In [10]:
# Remove emojis from the main messages
def remove_emoji(text):
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002500-\U00002BEF"  # chinese char
                            u"\U00002702-\U000027B0"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            u"\U0001f926-\U0001f937"
                            u"\U00010000-\U0010ffff"
                            u"\u2640-\u2642" 
                            u"\u2600-\u2B55"
                            u"\u200d"
                            u"\u23cf"
                            u"\u23e9"
                            u"\u231a"
                            u"\ufe0f"                 # dingbats
                            u"\u3030"
                                    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['cleanMsg'] = df['cleanMsg'].apply(lambda x: remove_emoji(x) if pd.isnull(x)==False else np.nan)

In [11]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    tags = re.compile('<.*?>')
    return re.sub(tags, '', text)

df['cleanMsg'] = df['cleanMsg'].apply(lambda x: remove_html_tags(x) if pd.isnull(x)==False else np.nan)

In [12]:
df.head()

,Platform,SocialNetwork,SenderScreenName,Message,CreatedTime,Sentiment,Date,hashtag,mention,url,Covid_Keywords,Covid_Keywords_Count,Covid_Flag,cleanMsg
0,FB - Wall Posts,FACEBOOK,Ann Heacock Sinton,Why isn't the online ordering working? I went...,2020-03-23 17:08:33,NEGATIVE,2020-03-23,[],[],[],{},0,0,Why isn't the online ordering working? I went...
1,FB - Wall Posts,FACEBOOK,Ari Graves,This is just greedy. $3.50 for this? Making mo...,2020-03-19 01:33:58,NEGATIVE,2020-03-19,[],[],[],{},0,0,This is just greedy. $3.50 for this? Making mo...
2,FB - Wall Posts,FACEBOOK,BL Schafer,Your company really needs an app like Sams Clu...,2020-03-18 17:35:08,NEGATIVE,2020-03-18,[],[],[],"{corona, virus}",2,1,Your company really needs an app like Sams Clu...
3,FB - Wall Posts,FACEBOOK,Bob Galivan,I was in the Beachwood store yesterday. NOT ON...,2020-03-21 12:28:26,NEGATIVE,2020-03-21,[],[],[],"{mask, covid, virus, glove}",4,1,I was in the Beachwood store yesterday. NOT ON...
4,FB - Wall Posts,FACEBOOK,Bob Varisco,I posted Wednesday about the fact that I'm at ...,2020-03-27 16:29:03,NEGATIVE,2020-03-27,[],[],[],{},0,0,I posted Wednesday about the fact that I'm at ...
